# Order parameters

## Graph structure

All edges are stored in a list of sets $edgelist$ such that each of them has a label corresponding to its position. At the same position in the list $X$ are stored the order parameters into a list of $8$ elements. Test: $X$ is now a dictionary

1) ring participation
Two integers: 0 if no ring, number of ring members if a closed ring
(0, 4,5,6,7... 12)

2) Other rings that p1, and p2 are a part of: 1 integer for p1, 1 for p2. Since there is only up to one extra ring per particle

3) number of bonded neighbors of the two particles (p1, p2) involved
Two integers: (1,2,3)

4) Local connectivity parameters
- Average number of bonds per neighbor of p1 and p2. (2 real number)

In [1]:
import numpy as np
#from matplotlib import *
from matplotlib import pyplot as plt
from heapq import *
from collections import deque
from collections import defaultdict
import copy

def adjacencyM(N, E):
    G=np.zeros([N, N])
    for e in E:
        G[int(e[0]), int(e[1])] =1
    return G

def rescale(x, M):
    if x>M/2:
        return rescale(x-M, M)
    elif x<-M/2:
        return rescale(x+M, M)
    else:
        return x
    
        
#G_aM = adjacencyM(N, aev[:,:-1])

In [109]:
def dijkstra(G, s, t):
    if s == t:
        return (0, deque())
    else:
        G[s].remove(t)
        G[t].remove(s) #to avoid the (s,t) edge
    S = set()
    q=[(0,s)]
    p=[v for v in range(len(G))]
    D = [float('Inf') for v in range(len(G))]; D[s] = 0
    while q:
        (dv, v) = heappop(q)
        #print(v)
        if v not in S:
            S.add(v)
            if v == t:
                G[s].add(t)
                G[t].add(s)
                return (D[t], getpath(p,s,t))
            #print(G[v])
            for u in G[v]:
                if u in S: continue
                d=dv+1
                if D[u] > d:
                    D[u] = d
                    p[u]=v
                    heappush(q, (d, u))
                    #print(">>"+ str(q))
                    
    G[s].add(t)
    G[t].add(s)
    return (-1, deque())


def getpath(p, s, t):
    v=t
    path=deque([t])
    while v!=s:
        v=p[v]
        path.appendleft(v)
    return path

In [3]:
l = 0
N=484
C=[]
with open("test.patch", "r") as snap:   #particle positions
    for line in snap:
        if l==0:
            N = int(line[1:])
            l=l+1
        elif l==1:
            tmp=[x for x in line.split()]
            xsize, ysize = float(tmp[0]), float(tmp[1])
            l=l+1
        else:
            tmp=[x for x in line.split()]
            #C.append([rescale(float(tmp[1]), xsize), rescale(float(tmp[2]), ysize)])
            C.append([float(tmp[1]), float(tmp[2])])
            
l = 0
events=[]
with open("breaklist_80.500.dat", "r") as file:  #Breaking Times
    for line in file:
        if l==0:
            l=l+1
        elif l==1:
            tmp=[x for x in line.split()]
            time = float(tmp[-1])
            l=l+1
        else:
            tmp=[x for x in line.split()]
            events.append([int(tmp[0]), int(tmp[1]), float(tmp[3])])

events=np.array(events)
C=np.array(C)

In [132]:
G=[set() for i in range(N)]
edgelist=deque()
y=[]
#X={}

for evs in events:
    ev = evs[:-1]
    edgelist.append(set(ev))
    #X[frozenset(ev)] = -1*np.ones(8)
    G[int(ev[0])].add(int(ev[1]))
    G[int(ev[1])].add(int(ev[0]))
    y.append(evs[-1])
    
X = [-1*np.ones(8) for i in range(len(edgelist))]
y = time/np.array(y)
m = np.mean(y)
std = np.std(y)
y=(((y-m)/std)+1)/2

In [134]:
avg=np.zeros(N)
for i in range(N):
    avg = 0
    if len(G[i]) == 0: continue
    for j in G[i]:
        e_idx = edgelist.index({i,j})  
        
        r1,p1 =dijkstra(G, i, j)  #finds 1.1

        r1 += 1
        if r1 != 0:
            r3 =dijkstra(G, i, p1[1])[0]+1;
            
            r4 =dijkstra(G, j, p1[-2])[0]+1  #finds 4
            
            m=int(len(p1)/2)
            

            G[p1[m+1]].remove(p1[m]); G[p1[m]].remove(p1[m+1])

            r2 =dijkstra(G, i, j)[0] + 1 #finds 1.2
            G[p1[m+1]].add(p1[m]); G[p1[m]].add(p1[m+1])
            
            X[e_idx][0:4] = np.array([r1, r2, r3, r4])
        else:
            X[e_idx][0:4] = np.zeros(4)
        X[e_idx][4:6] = np.array([len(G[i]), len(G[j])])
        avg += len(G[j])
        
    avg /= len(G[i])                                                  #finds 3
    
    for j in G[i]:
        idx= 7 if i>j else 6
        X[edgelist.index({i,j})][idx] = avg
            

## $X$ as dictionary

In [112]:
avg=np.zeros(N)
for i in range(N):
    avg = 0
    if len(G[i]) == 0: continue
    for j in G[i]:
        #print(j)
        #tmp = copy.deepcopy(G)
        #tmp[i].remove(j); tmp[j].remove(i)
        r1,p1 =dijkstra(G, i, j)  #finds 1.1

        r1 += 1
        if r1 != 0:
            r3 =dijkstra(G, i, p1[1])[0]+1;
            
            r4 =dijkstra(G, j, p1[-2])[0]+1  #finds 4
            
            m=int(len(p1)/2)
            

            G[p1[m+1]].remove(p1[m]); G[p1[m]].remove(p1[m+1])

            r2 =dijkstra(G, i, j)[0] + 1 #finds 1.2
            G[p1[m+1]].add(p1[m]); G[p1[m]].add(p1[m+1])
            
            X[frozenset({i,j})][0:4] = np.array([r1, r2, r3, r4])
            #if X[frozenset({0,36})][0]==-1: X[frozenset({0,36})][0]=r1
            #if X[frozenset({0,36})][0]==-1: X[frozenset({0,36})][0]=r1
        else:
            X[frozenset({i,j})][0:4] = np.zeros(4)
        X[frozenset({i,j})][4:6] = np.array([len(G[i]), len(G[j])])
        avg += len(G[j])
        #idx = edgelist.index({i,j})  
    avg /= len(G[i])                                                  #finds 3
    
    for j in G[i]:
        idx= 7 if i>j else 6
        X[frozenset({i,j})][idx] = avg
            